<a href="https://colab.research.google.com/github/Moore-Lab-UMass/ENCODE-API/blob/main/Pull-Histone-Bams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Jill E. Moore
#Moore Lab - UMass Chan Medical School
#May 2022

import os
import json
import urllib
from urllib import request, parse, error
import requests

In [2]:
def Download_File(name, extension, outputDir): #downloads file to local machine
    if not os.path.exists(outputDir+"/"+name+"."+extension):
        url="https://www.encodeproject.org/files/"+name+"/@@download/"+name+"."+extension
        r = requests.get(url)
        outputFile=open(outputDir+"/"+name+"."+extension, "wb")
        outputFile.write(r.content)
        outputFile.close()

In [3]:
def Extract_Experiment_Metatdata(exp, genome): #extracts experiment metadata
  urlExp = "https://www.encodeproject.org/experiments/"+exp+"/?format=json"
  response = urllib.request.urlopen(urlExp)
  data = json.loads(response.read())

  rfa = data["award"]["rfa"] #project/phase of encode
  lab = data["lab"]["title"] #lab that submitted the experiment
  biosample = data["biosample_ontology"]["term_name"] #short name of biosample
  tf = data["target"]["label"] #target of the assay, i.e. TF

  bam = "NA"
  numReads = "NA"
  replicate = "NA"

  for entry in data["files"]: #loops through files associated with experiment
    if entry["file_type"] == "bam" and entry["status"] == "released" \
      and entry["assembly"] == genome and entry["output_type"] == "alignments":

      bam = entry["accession"] #bam file
      #Download_File(bam, "bam", "/data/output")
      replicate = ",".join([str(i) for i in entry["biological_replicates"]])

      for qc in entry["quality_metrics"]:
        if "mapped_reads" in qc:
          numReads = qc["mapped_reads"]

      for pipeline in entry["analyses"]:
        if pipeline["status"] == "released":
          print(exp + "\t" + bam +"\t" + replicate + "\t" + biosample + "\t"+ \
                tf +"\t" + rfa +"\t"+ lab + "\t" + str(numReads))


In [4]:
## Parameters
genome = "GRCh38"
species = "Homo+sapiens"

## Build query
urlMain = "https://www.encodeproject.org/search/?type=Experiment&status=released" + \
    "&perturbed=false&perturbed=true&assay_title=Histone+ChIP-seq" + \
    "&assay_title=Mint-ChIP-seq" + \
    "&replicates.library.biosample.donor.organism.scientific_name=" + species + \
    "&format=json&limit=50" #update limit=all for all datasets

response = urllib.request.urlopen(urlMain)
data = json.loads(response.read())

print("experiment_accession" + "\t" + "bam_accession" + "\t" + "replicate" + \
      "\t" + "biosample" + "\t" + "tf" + "\t" + "rfa" +"\t"+ "lab" + "\t" + \
      "number_reads")

for entry in data["@graph"]: #loops through experiments
  Extract_Experiment_Metatdata(entry["accession"], genome)

experiment_accession	bam_accession	replicate	biosample	tf	rfa	lab	number_reads
ENCSR976GAB	ENCFF342XID	1	Calu3	H3K27ac	ENCODE4	Bradley Bernstein, Broad	50938672
ENCSR995MHN	ENCFF621ZWN	1	Caco-2	H3K4me3	ENCODE4	Bradley Bernstein, Broad	42961494
ENCSR571QQB	ENCFF541JYL	1	Caco-2	H3K27ac	ENCODE4	Bradley Bernstein, Broad	35006584
ENCSR291QUA	ENCFF249DZS	1	MCF 10A	H3K4me3	ENCODE4	Bradley Bernstein, Broad	30850856
ENCSR685JSL	ENCFF074XGA	1	MCF 10A	H3K27ac	ENCODE4	Bradley Bernstein, Broad	31046422
ENCSR930EMO	ENCFF611ZHJ	1	HCT116	H3K27ac	ENCODE4	Bradley Bernstein, Broad	66419384
ENCSR993EBE	ENCFF961QXY	1	HCT116	H3K4me1	ENCODE4	Bradley Bernstein, Broad	143018614
ENCSR280DQH	ENCFF597ZTM	1	HCT116	H3K4me3	ENCODE4	Bradley Bernstein, Broad	47101682
ENCSR253KLE	ENCFF140ENA	1	HCT116	H3K36me3	ENCODE4	Bradley Bernstein, Broad	118370392
ENCSR728KQX	ENCFF535QKX	1	HCT116	H3K27ac	ENCODE4	Bradley Bernstein, Broad	70872432
ENCSR523IAZ	ENCFF264RRX	1	HCT116	H3K4me3	ENCODE4	Bradley Bernstein, Broad	58647562
ENCS